In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.cluster import KMeans

In [3]:
# Load dataset
df = pd.read_csv('yourdataset.csv')  # Replace with your dataset file path

In [4]:
# Prepare the data
X = df.drop(['target'], axis="columns")  # Replace 'target' with the name of the target column
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [5]:
# Calculate feature correlations with target
correlations = X_train.corrwith(y_train)

In [6]:
# Clustering features using K-means
num_dimensions = 2  # Number of dimensions or clusters
correlations_reshaped = np.reshape(correlations.values, (-1, 1))
kmeans = KMeans(n_clusters=num_dimensions, random_state=0).fit(correlations_reshaped)
clusters = kmeans.labels_

# Group features into clusters
groups = [np.where(clusters == i)[0] for i in range(num_dimensions)]

In [7]:
# Apply LDA to each feature cluster
features_lda = np.empty((X_train.shape[0], num_dimensions))
features_lda_test = np.empty((X_test.shape[0], num_dimensions))

for i, group in enumerate(groups):
    lda = LDA(n_components=1)
    features_lda[:, i] = lda.fit_transform(X_train.iloc[:, group], y_train).ravel()
    features_lda_test[:, i] = lda.transform(X_test.iloc[:, group]).ravel()

In [8]:
# Normalize the new features for QML
minmax_scaler = MinMaxScaler().fit(features_lda)
X_train_qml = minmax_scaler.transform(features_lda)
X_test_qml = minmax_scaler.transform(features_lda_test)